In [ ]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import csv
import re
# import nltk
# from dask.distributed import Client
# client = Client(processes=False)
# nltk.download()
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/Amazon Dataset task 1/dataset/train.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)[["TITLE", "BRAND","DESCRIPTION","BULLET_POINTS", "BROWSE_NODE_ID"]]

In [ ]:
data.BULLET_POINTS[1]

'[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]'

In [ ]:
data = data[data.groupby('BROWSE_NODE_ID')['BROWSE_NODE_ID'].transform('count').ge(500)]

In [ ]:
data.isna().sum()

TITLE                 64
BRAND              42518
DESCRIPTION       605358
BULLET_POINTS     122539
BROWSE_NODE_ID         0
dtype: int64

In [ ]:
data = data.fillna("")
data.reset_index(drop=True)

,TITLE,BRAND,DESCRIPTION,BULLET_POINTS,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",MerryMakers,"Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker,The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,1
2,Teacher Planner Company A4 6 Lesson Academic T...,,,,4
3,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Bhavya Enterprise,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",5
4,Glance Women's Wallet (Black) (LW-21),Glance,This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,6
...,...,...,...,...,...
2357831,Holy Heart Collections Supremo Zig Zag Line Pr...,Holy Heart Collections,Holy Heart Collections presents flower print b...,"[Color : Multicolor, Material : Fine Polyester...",739
2357832,GVND 6 Pack Abs Exercise Machine with Multi Pu...,Gvnd,<p><strong>GVND Six pack Abs is the revolution...,[Multiple Exercise Modes: This Abdominal Train...,1835
2357833,Premium Aviator Sunglasses - HD Polarized (Bri...,Generic,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",1040
2357834,Social Distance Stickers - Set of 5 Sticker Sl...,Generic,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,15199


In [ ]:
data.BULLET_POINTS = data.BULLET_POINTS.astype(str) 
data.BRAND = data.BRAND.astype(str)
data.TITLE = data.TITLE.astype(str)
data.DESCRIPTION = data.DESCRIPTION.astype(str)

In [ ]:
data['combined'] = data.TITLE+" "+data.DESCRIPTION+" "+data.BRAND+" "+data.BULLET_POINTS

In [ ]:
data.combined[0]

"Pete The Cat Bedtime Blues Doll, 14.5 Inch Pete the Cat is the coolest, most popular cat in town. The new Pete the Cat Bedtime Blues Doll from MerryMakers rocks his striped pj's and red slippers. He is one sleepy cat and is ready to cuddle. Measures 14.5 inches tall. Safe for all ages. Removable clothing. Surface wash only. New 2015 MerryMakers [Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]"

In [ ]:
data.reset_index(drop=True)

,TITLE,BRAND,DESCRIPTION,BULLET_POINTS,BROWSE_NODE_ID,combined
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",MerryMakers,"Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch Pet..."
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker,The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
2,Teacher Planner Company A4 6 Lesson Academic T...,,,,4,Teacher Planner Company A4 6 Lesson Academic T...
3,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Bhavya Enterprise,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...
4,Glance Women's Wallet (Black) (LW-21),Glance,This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,6,Glance Women's Wallet (Black) (LW-21) This Bla...
...,...,...,...,...,...,...
2357831,Holy Heart Collections Supremo Zig Zag Line Pr...,Holy Heart Collections,Holy Heart Collections presents flower print b...,"[Color : Multicolor, Material : Fine Polyester...",739,Holy Heart Collections Supremo Zig Zag Line Pr...
2357832,GVND 6 Pack Abs Exercise Machine with Multi Pu...,Gvnd,<p><strong>GVND Six pack Abs is the revolution...,[Multiple Exercise Modes: This Abdominal Train...,1835,GVND 6 Pack Abs Exercise Machine with Multi Pu...
2357833,Premium Aviator Sunglasses - HD Polarized (Bri...,Generic,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",1040,Premium Aviator Sunglasses - HD Polarized (Bri...
2357834,Social Distance Stickers - Set of 5 Sticker Sl...,Generic,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,15199,Social Distance Stickers - Set of 5 Sticker Sl...


In [ ]:
data.head(10)

,TITLE,BRAND,DESCRIPTION,BULLET_POINTS,BROWSE_NODE_ID,combined
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",MerryMakers,"Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch Pet..."
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker,The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
4,Teacher Planner Company A4 6 Lesson Academic T...,,,,4,Teacher Planner Company A4 6 Lesson Academic T...
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Bhavya Enterprise,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...
6,Glance Women's Wallet (Black) (LW-21),Glance,This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,6,Glance Women's Wallet (Black) (LW-21) This Bla...
9,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...,Chapin International,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...,,9,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...
11,White : Braun KF400-WH Aromaster 10-Cup Coffee...,Braun,Color:White Product description 10 Cup Carafe ...,"[Water-dispersal system provides fast, uniform...",11,White : Braun KF400-WH Aromaster 10-Cup Coffee...
13,Microsoft Sidewinder Gamepad Pro,Microsoft,,Dual-function D-Pad for comfortable control - ...,13,Microsoft Sidewinder Gamepad Pro Microsoft Du...
16,OXO Good Grips BPA-Free Turkey Baster with Cle...,OXO,<b>OXO Good Grips Turkey Baster</b><br><br><ul...,[Unique bulb shape and collar for two comforta...,16,OXO Good Grips BPA-Free Turkey Baster with Cle...
19,"InterDesign 14 by 9 by 5-Inch Basic Basket, Me...",InterDesign,Plastic basket with chrome handle. You can sto...,"[Made of extra durable translucent plastic,Chr...",19,"InterDesign 14 by 9 by 5-Inch Basic Basket, Me..."


In [ ]:
data=data.reset_index(drop=True)

In [ ]:
data.head(10)

,TITLE,BRAND,DESCRIPTION,BULLET_POINTS,BROWSE_NODE_ID,combined
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",MerryMakers,"Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch Pet..."
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker,The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
2,Teacher Planner Company A4 6 Lesson Academic T...,,,,4,Teacher Planner Company A4 6 Lesson Academic T...
3,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Bhavya Enterprise,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...
4,Glance Women's Wallet (Black) (LW-21),Glance,This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,6,Glance Women's Wallet (Black) (LW-21) This Bla...
5,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...,Chapin International,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...,,9,Chapin 1380 Premier Pro 3-Gallon Tri-Poxy Stee...
6,White : Braun KF400-WH Aromaster 10-Cup Coffee...,Braun,Color:White Product description 10 Cup Carafe ...,"[Water-dispersal system provides fast, uniform...",11,White : Braun KF400-WH Aromaster 10-Cup Coffee...
7,Microsoft Sidewinder Gamepad Pro,Microsoft,,Dual-function D-Pad for comfortable control - ...,13,Microsoft Sidewinder Gamepad Pro Microsoft Du...
8,OXO Good Grips BPA-Free Turkey Baster with Cle...,OXO,<b>OXO Good Grips Turkey Baster</b><br><br><ul...,[Unique bulb shape and collar for two comforta...,16,OXO Good Grips BPA-Free Turkey Baster with Cle...
9,"InterDesign 14 by 9 by 5-Inch Basic Basket, Me...",InterDesign,Plastic basket with chrome handle. You can sto...,"[Made of extra durable translucent plastic,Chr...",19,"InterDesign 14 by 9 by 5-Inch Basic Basket, Me..."


In [ ]:
X = data.combined
y = data.BROWSE_NODE_ID

In [ ]:
# from gensim.parsing.preprocessing import remove_stopwords as rs

# data.TITLE = data["TITLE"].apply(lambda x: rs(x))

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10).split(X, y)
_, train_idx = next(skf)
_, test_idx = next(skf)

In [ ]:
train_data, y_train = X[train_idx], y[train_idx]
val_data, y_val = X[test_idx], y[test_idx]

In [ ]:
y_val.unique().sum()

2498087

In [ ]:
train_data = train_data.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
vectorizer = feature_extraction.text.CountVectorizer(max_features=4000, ngram_range=(1,2))
corpus = train_data.astype(str)
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

In [ ]:
#from sklearn.ensemble import BaggingClassifier
#clf = BaggingClassifier(n_jobs=10, max_samples=1.0 / 14, n_estimators=14).fit(X_title_tfid, y_train)

classifier = naive_bayes.MultinomialNB()

#from sklearn.naive_bayes import MultinomialNB
#classifier = MultinomialNB()
#clf = classifier.fit(X_title_tfid, y_train)

#from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier( random_state=0, n_jobs=10).fit(X_title_tfid, y_train)

In [ ]:
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
val_data = val_data.fillna("")

In [ ]:
predicted = model.predict(val_data)

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_val, predicted)
accuracy

0.638771078614325

In [ ]:
import pickle
model = pickle.dump(clf,open('/content/drive/MyDrive/Amazon Challenge/modelMultinomial_BULLET_BRAND.pkl','wb'))

In [ ]:
import numpy as np
import csv
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
clf = pickle.load(open('/content/drive/MyDrive/Amazon Challenge/modelMultinomial.pkl','rb'))

In [ ]:
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X_title_val = countvect.transform(val_data)
X_title_tfid_val = tfidf_transformer.transform(X_title_val)

#clf.score(X_title_tfid_val,y_val)

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/cleaned_test_all_all.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
test_data

,Unnamed: 0,PRODUCT_ID,combined_clean,combined_clean_all,combined_clean_all_all
0,0,1,command 3m small kitchen hook white decor dama...,command 3m small kitchen hook white decor dama...,command 3m small kitchen hook white decor dama...
1,1,2,oneal jump hardwar jag unisexadult glove black...,oneal jump hardwar jag unisexadult glove black...,oneal jump hardwar jag unisexadult glove black...
2,2,3,nfl detroit lion portabl parti fridg 158 quart...,nfl detroit lion portabl parti fridg 158 quart...,nfl detroit lion portabl parti fridg 158 quart...
3,3,4,panason singl line kxts880mx cord phone white ...,panason singl line kxts880mx cord phone white ...,panason singl line kxts880mx cord phone white ...
4,4,5,zero babi girl 100 cotton innerwear bloomer dr...,zero babi girl 100 cotton innerwear bloomer dr...,zero babi girl 100 cotton innerwear bloomer dr...
...,...,...,...,...,...
110770,110770,110771,aahna e mall oneblad hybrid trimmer shaver edg...,aahna e mall oneblad hybrid trimmer shaver edg...,aahna e mall oneblad hybrid trimmer shaver edg...
110771,110771,110772,grin health n99 anti pollut reusabl washabl ma...,grin health n99 anti pollut reusabl washabl ma...,grin health n99 anti pollut reusabl washabl ma...
110772,110772,110773,asian armi pink ultra reusabl respir cloth mas...,asian armi pink ultra reusabl respir cloth mas...,asian armi pink ultra reusabl respir cloth mas...
110773,110773,110774,im safe 3 pli nonwoven dispos surgic face mask...,im safe 3 pli nonwoven dispos surgic face mask...,im safe 3 pli nonwoven dispos surgic face mask...


In [ ]:
test_data['combined_clean_all_all']= test_data.combined_clean_all_all.astype(str)

In [ ]:
test_data.isna().sum()
test_data = test_data.fillna("")

In [ ]:
test_data.BULLET_POINTS = test_data.BULLET_POINTS.astype(str)
test_data.BRAND = test_data.BRAND.astype(str)
test_data.TITLE = test_data.TITLE.astype(str)
test_data.DESCRIPTION = test_data.DESCRIPTION.astype(str)

In [ ]:
test_data['combined'] = test_data.TITLE + " " + test_data.BULLET_POINTS+ " " + test_data.BRAND

In [ ]:
test_data.combined[1]

"O'Neal Jump Hardware JAG Unisex-Adult Glove (Black/Red, 11) Synthetic leather palm with double-layer thumb reinforcement. Hook-and-loop adjustable wrist closure."

In [ ]:
sub = pd.DataFrame(columns=['PRODUCT_ID', 'BROWSE_NODE_ID'])

In [ ]:
sub['PRODUCT_ID'] = test_data['PRODUCT_ID']

In [ ]:
sub['BROWSE_NODE_ID'] = model.predict(test_data.combined_clean_all_all)

In [ ]:
sub['BROWSE_NODE_ID'] = model.predict(test_data.combined)

In [ ]:
sub

,PRODUCT_ID,BROWSE_NODE_ID
0,1,1140
1,2,2321
2,3,115
3,4,604
4,5,1239
...,...,...
110770,110771,4368
110771,110772,1551
110772,110773,1551
110773,110774,13568


In [ ]:
sub.to_csv('/content/drive/MyDrive/Amazon Challenge/submissionfinal10.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer(ngram_range=(1,2), max_features=20000, stop_words='english')
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [ ]:
#10001 then gaps of 10000
X_title_test = countvect.transform(test_data.combined)
X_title_tfid_test = tfidf_transformer.transform(X_title_test)

sub['BROWSE_NODE_ID'] = clf.predict(X_title_tfid_test)

In [ ]:
sub

,PRODUCT_ID,BROWSE_NODE_ID
0,1,413
1,2,1186
2,3,11
3,4,4
4,5,1922
...,...,...
110770,110771,4
110771,110772,800
110772,110773,800
110773,110774,800


In [ ]:
sub.to_csv('/content/drive/MyDrive/Amazon Challenge/submissionfinal7.csv', index=False)


In [ ]:
data1 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/Jsub1.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data8 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/Ssub1.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data10 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/Ssub2.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data11 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/submissionfinal12.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data5 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/test Submissions/submissionfinal1.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data6 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/test Submissions/submissionfinal6.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data7 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/test Submissions/submissionfinal7.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data8 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/test Submissions/submissionfinal8.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)
data10 = pd.read_csv("/content/drive/MyDrive/Amazon Challenge/test Submissions/submissionfinal10.csv", escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
for i in range(110774):
  dict = {}
  dict[data1.BROWSE_NODE_ID[i]]=0
  dict[data8.BROWSE_NODE_ID[i]]=0
  dict[data10.BROWSE_NODE_ID[i]]=0
  dict[data11.BROWSE_NODE_ID[i]]=0
  dict[data5.BROWSE_NODE_ID[i]]=0
  dict[data6.BROWSE_NODE_ID[i]]=0
  dict[data7.BROWSE_NODE_ID[i]]=0
  dict[data8.BROWSE_NODE_ID[i]]=0
  dict[data10.BROWSE_NODE_ID[i]]=0
  dict[data1.BROWSE_NODE_ID[i]]=dict[data1.BROWSE_NODE_ID[i]]+1
  dict[data8.BROWSE_NODE_ID[i]]=dict[data8.BROWSE_NODE_ID[i]]+1
  dict[data10.BROWSE_NODE_ID[i]]=dict[data10.BROWSE_NODE_ID[i]]+1
  dict[data11.BROWSE_NODE_ID[i]]=dict[data11.BROWSE_NODE_ID[i]]+1
  dict[data5.BROWSE_NODE_ID[i]]=dict[data5.BROWSE_NODE_ID[i]]+1
  dict[data6.BROWSE_NODE_ID[i]]=dict[data6.BROWSE_NODE_ID[i]]+1
  dict[data7.BROWSE_NODE_ID[i]]=dict[data7.BROWSE_NODE_ID[i]]+1
  dict[data8.BROWSE_NODE_ID[i]]=dict[data8.BROWSE_NODE_ID[i]]+1
  dict[data10.BROWSE_NODE_ID[i]]=dict[data10.BROWSE_NODE_ID[i]]+1
  ans=0
  val=""
  for j in dict:
    if dict[j]>ans:
      ans=dict[j]
      val=j
  
  data1.BROWSE_NODE_ID[i]=val

In [ ]:
print(data1.BROWSE_NODE_ID[0])
print(data8.BROWSE_NODE_ID[0])
print(data10.BROWSE_NODE_ID[0])
print(data11.BROWSE_NODE_ID[0])
print(data5.BROWSE_NODE_ID[0])
print(data6.BROWSE_NODE_ID[0])
print(data7.BROWSE_NODE_ID[0])
print(data8.BROWSE_NODE_ID[0])
print(data10.BROWSE_NODE_ID[0])

In [ ]:
data1.to_csv('/content/drive/MyDrive/Amazon Challenge/submissionfinal14.csv', index=False)